In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

In [3]:
model_id = "beomi/Llama-3-Open-Ko-8B-Instruct-preview"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [9]:
emotion_list = [('Joy', 0.7897149510681629), ('Disgust', 0.10675244318554178), ('Sadness', 0.07509662717347965), ('Anger', 0.01339526695664972), ('Fear', 0.009518425795249641), ('Surprise', 0.005522291059605777)]
keyword_list = ['목소리' '미소' '행복' '온도' '자장가' '슬플' '힘들' '바라' '눈물' '손길']
color_list = [['Joy', 'green'], ['Joy', 'pink'], ['Joy', 'light blue'], ['Joy', 'orange']]

In [10]:
emotion_text = ''
for item in emotion_list:
  emo = item[0]
  score = item[1]
  result = f'{item[0]}: {item[1]:.3f}, '
  emotion_text += result
emotion_text = emotion_text.strip()
keyword_text = ', '.join(keyword_list)
colors = []
for color in color_list:
  colors.append(color[1])
colors = list(set(colors))
color_text = ', '.join(colors)

In [ ]:
messages = [
    {"role": "system", "content": "You are an illustration creation AI that can create a scene in a very sensuous and emotional way. Express complex emotions in detail based on emotion ratio, keyword details, and color details. and describe it in 450 characters or less."},
    {"role": "user", "content": f"The emotion ratio is '{emotion_text}', the keyword is '{keyword_text}', and finally the color is '{color_text}'. Based on this, think of a scene or illustration and describe it."},
]


input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=512,
    eos_token_id=terminators,
    do_sample=True,
    temperature=1,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
